# **Quickly query a local SQL Server database from Jupyter Notebook**

*   **Connect to an SQL database**: There are several Python libraries that can help connect with an SQL database, but we're going to go with [Pyodbc](https://pypi.org/project/pyodbc/), as the official documentation on the Pyodbc website is very clear and helpful.

In [ ]:
# importing our main modules
import pandas as pd
import pyodbc

In [ ]:
# how to connect to a local SQL server database https://datatofish.com/how-to-connect-python-to-sql-server-using-pyodbc/
def getSQLServerConnection(server_address,database_name):
    conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                          f"Server={server_address};"
                          f"Database={database_name};"
                          "Trusted_Connection=yes;")
    return conn

# connecting to my local SQL Server entity
conn = getSQLServerConnection("********", 
                              "******"
                             )

In [ ]:
# we can now pass the data into a Pandas dataframe, using native T-SQL language
def getQuery(sql_query):
    df = pd.read_sql_query(sql_query,conn)
    return df

# testing our newly created function
df = getQuery(
"""
SELECT TOP 3 *
FROM sales
"""
    )
df

,salesid,listid,sellerid,buyerid,eventid,dateid,quantitysold,pricepaid,commission,saleday,saletime
0,102262,116982,9971,9814,1316,2013,1,287,43.05,6/7/08,09:24:36
1,102263,116983,40052,167,8592,2038,1,79,11.85,7/31/2008,09:26:26
2,102264,116984,29567,24492,4999,1894,2,952,142.80,9/3/08,09:27:44


# **Examples**

In [ ]:
# 1. Overall total value and volume.
df = getQuery(
"""
SELECT
    FORMAT(COUNT(salesid), 'N0') AS total_sales,
    FORMAT(SUM(pricepaid), 'N0') AS total_value
FROM
    sales;
"""
    )
df

,total_sales,total_value
0,"172,456","110,765,431"


In [ ]:
# 2. Top 5 sellers by value (user name, first name, last name, user name, value and quantity sold) in New York.
df = getQuery(
"""
WITH t2 (userid, username, firstname, lastname, state) AS
    (SELECT userid, username, firstname, lastname, state FROM allusers)
SELECT TOP 5
    t2.username,
    t1.sellerid,
    CONCAT(t2.firstname, ' ', t2.lastname) AS full_name,
    SUM(t1.pricepaid) AS total_value,
    SUM(CAST(t1.quantitysold AS INT)) AS total_volume
FROM
    sales AS t1
    LEFT JOIN t2 on t1.sellerid = t2.userid
WHERE
    t2.state = 'NY'
GROUP BY 
    t2.username,t1.sellerid,CONCAT(t2.firstname, ' ', t2.lastname)
ORDER BY
    total_value DESC;
"""
    )
df

,username,sellerid,full_name,total_value,total_volume
0,UQD29HRW,41337,Lillith Curtis,24223,22
1,HSR02WON,21492,Venus Maddox,14281,35
2,AIG24AZS,17034,Jeremy Reilly,13920,6
3,YYJ75QFH,33128,Amaya Petersen,13041,7
4,YRT41PVD,27369,Stuart Bentley,12677,7


In [ ]:
# 3. Top 5 buyers (user name, first name, last name, user name, value and quantity sold) overall by value and volume.
df = getQuery(
"""
WITH t2 (userid, username, firstname, lastname, state) AS
    (SELECT userid, username, firstname, lastname, state FROM allusers)
SELECT TOP 5
    t2.username,
    t1.buyerid,
    CONCAT(t2.firstname, ' ', t2.lastname),
    SUM(t1.pricepaid) AS total_value,
    SUM(CAST(t1.quantitysold AS INT)) AS total_volume
FROM
    sales AS t1
    LEFT JOIN t2 on t1.buyerid = t2.userid
GROUP BY 
    t2.username,t1.buyerid,CONCAT(t2.firstname, ' ', t2.lastname)
ORDER BY
    total_value DESC;
"""
    )
df

,username,buyerid,,total_value,total_volume
0,LXF16RPG,4303,Latifah Wade,46710,33
1,YHF13OYW,240,Charity Dominguez,46570,48
2,BWV25SNA,1239,Chancellor Odonnell,45684,52
3,CKM68QQM,3286,Ava Marquez,45414,38
4,KUP01TZN,16391,Aretha Mason,40943,38


In [ ]:
# 4. Top 5 venues overall by capacity (venue seats) and what is their seating capacity?
df = getQuery(
"""
SELECT TOP 5
    venuename,
    FORMAT(CAST(venueseats AS int), 'N0') AS capacity
FROM
    venue
ORDER BY
    venueseats DESC;
"""
    )
df

,venuename,capacity
0,FedExField,"91,704"
1,New York Giants Stadium,"80,242"
2,Arrowhead Stadium,"79,451"
3,INVESCO Field,"76,125"
4,Dolphin Stadium,"74,916"


In [ ]:
# 5. Most expensive ticket sold (price per ticket), who (buyer’s first & last name) bought this ticket? Who sold the ticket (seller’s first & last name)? What was the event name and in which city was it hosted?
df = getQuery(
"""
SELECT TOP 5
    t1.priceperticket,
    t2.eventid,
    CONCAT(ISNULL(t4.firstname, 'unknown'), ' ', t4.lastname) AS buyer_name,
    CONCAT(ISNULL(t5.firstname, 'unknown'), ' ', t5.lastname) AS seller_name,
    t2.eventname,
    t6.venuecity
FROM
    listings AS t1
    LEFT JOIN allevents AS t2 ON t1.eventid = t2.eventid
    LEFT JOIN sales AS t3 ON t2.eventid = t3.eventid
    LEFT JOIN allusers AS t4 ON t3.buyerid = t4.userid
    LEFT JOIN allusers AS t5 ON t3.sellerid = t5.userid
    LEFT JOIN venue AS t6 ON t2.venueid = t6.venueid
ORDER BY
    t1.priceperticket DESC;
"""
    )
df

,priceperticket,eventid,buyer_name,seller_name,eventname,venuecity
0,2500,3504,Barrett Lynn,Neville Myers,The Cherry Orchard,New York City
1,2500,3504,Clayton Chase,Neville Myers,The Cherry Orchard,New York City
2,2500,3504,Cara Hampton,Neville Myers,The Cherry Orchard,New York City
3,2500,3504,Grady Oneil,Kai Hooper,The Cherry Orchard,New York City
4,2500,3504,Alexa Reese,Cassidy Gentry,The Cherry Orchard,New York City
